In [1]:
!pip install fpdf2


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install streamlit plotly


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install ipywidgets plotly


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

In [5]:
files = {
    "Waze heavy traffic.csv": "Trafic dense",
    "Waze stand still traffic.csv": "Trafic à l’arrêt",
    "Waze accident minor.csv": "Accident léger",
    "Waze accident major.csv": "Accident grave",
    "Waze pot_hole.csv": "Nid-de-poule",
    "HAZARD_ON_ROAD_TRAFFIC_LIGHT_FAULT.csv": "Panne de feu tricolore",
    "HAZARD_WEATHER_FLOOD.csv": "Inondation"
}

In [6]:
def extract_lat_lon(point):
    if pd.isna(point):
        return pd.Series([None, None])
    point = point.replace("Point(", "").replace(")", "")
    lon, lat = point.split()
    return pd.Series([float(lat), float(lon)])


In [7]:
def clean_date(date):
    if pd.isna(date):
        return pd.NaT

    date = str(date).lower()
    date = date.replace("avr.", "avril").replace("févr.", "février").replace("janv.", "janvier")
    date = date.replace("mars", "mars").replace("juin", "juin")
    date = date.replace("juil.", "juillet").replace("août", "août")
    date = date.replace("sept.", "septembre").replace("oct.", "octobre")
    date = date.replace("nov.", "novembre").replace("déc.", "décembre")

    return pd.to_datetime(date, errors="coerce", dayfirst=True)

In [8]:
dfs = []

for file, scenario in files.items():
    df = pd.read_csv(file, low_memory=False)
    df["scenario"] = scenario

    # Coordonnées
    df[["latitude", "longitude"]] = df["Location"].apply(extract_lat_lon)

    # Date
    df["Date"] = df["Date"].apply(clean_date)
    df["annee"] = df["Date"].dt.year

    # Ville inconnue
    df["City"] = df["City"].fillna("Inconnue")

    dfs.append(df)

waze = pd.concat(dfs, ignore_index=True)

C:\Users\bellc\AppData\Local\Temp\ipykernel_22484\3211778559.py:12: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(date, errors="coerce", dayfirst=True)
C:\Users\bellc\AppData\Local\Temp\ipykernel_22484\3211778559.py:12: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(date, errors="coerce", dayfirst=True)
C:\Users\bellc\AppData\Local\Temp\ipykernel_22484\3211778559.py:12: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(date, errors="coerce", dayfirst=True)


In [9]:
print(waze.shape)
waze[["City", "scenario", "annee"]].head()

(3558101, 11)


,City,scenario,annee
0,Inconnue,Trafic dense,NaN
1,Inconnue,Trafic dense,NaN
2,Inconnue,Trafic dense,NaN
3,Inconnue,Trafic dense,NaN
4,Inconnue,Trafic dense,NaN


In [10]:
villes = sorted(waze["City"].unique())
annees = sorted(waze["annee"].dropna().unique())

ville_widget = widgets.Dropdown(
    options=villes,
    description="Ville :",
    value=villes[0]
)

annee_widget = widgets.Dropdown(
    options=["Toutes"] + list(annees),
    description="Année :",
    value="Toutes"
)

display(ville_widget, annee_widget)

Dropdown(description='Ville :', options=('Antony', 'Arcueil', 'Athis-Mons', 'Bagneux', 'Bajolet (Forges-les-Ba…

Dropdown(description='Année :', options=('Toutes', np.float64(2021.0), np.float64(2022.0), np.float64(2023.0),…

In [11]:
def update_histogram(ville, annee):
    df = waze[waze["City"] == ville]

    if annee != "Toutes":
        df = df[df["annee"] == annee]

    stats = (
        df["scenario"]
        .value_counts()
        .reset_index()
        .rename(columns={"index": "Scénario", "scenario": "Nombre"})
    )

    fig = px.bar(
        stats,
        x="Scénario",
        y="Nombre",
        color="Scénario",
        text_auto=True,
        title=f"Événements Waze – {ville}"
    )

    fig.update_layout(
        xaxis_title="Scénario",
        yaxis_title="Nombre d'événements",
        showlegend=False
    )

    fig.show()

In [2]:
import streamlit as st
import pandas as pd
import plotly.express as px

# =========================
# CONFIG
# =========================
st.set_page_config(
    page_title="Dashboard Waze – Analyse par ville",
    layout="wide"
)

# =========================
# CHARGEMENT DES DONNÉES
# =========================
@st.cache_data
def load_data():
    return pd.read_csv("waze_all_clean.csv", parse_dates=["Date"])

waze = load_data()

# =========================
# SIDEBAR — FILTRES
# =========================
st.sidebar.title("Filtres")

villes = sorted(waze["City"].unique())
ville = st.sidebar.selectbox("Ville", villes)

annees = sorted(waze["annee"].dropna().unique())
annee = st.sidebar.selectbox("Année", ["Toutes"] + list(annees))

# =========================
# FILTRAGE
# =========================
df = waze[waze["City"] == ville]

if annee != "Toutes":
    df = df[df["annee"] == annee]

# =========================
# TITRE
# =========================
st.title(f"📊 Événements Waze – {ville}")

# =========================
# HISTOGRAMME SCÉNARIOS
# =========================
count_scenario = (
    df["scenario"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "Scénario", "scenario": "Nombre"})
)

fig = px.bar(
    count_scenario,
    x="Scénario",
    y="Nombre",
    color="Scénario",
    title="Répartition des scénarios",
    text_auto=True
)

fig.update_layout(
    xaxis_title="Scénario",
    yaxis_title="Nombre d'événements",
    showlegend=False
)

st.plotly_chart(fig, use_container_width=True)

# =========================
# TABLE DE DÉTAIL (OPTIONNEL)
# =========================
with st.expander("Voir les données détaillées"):
    st.dataframe(df)


2026-01-19 10:49:25.941 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-19 10:49:25.957 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-19 10:49:25.992 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-19 10:49:25.994 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


FileNotFoundError: [Errno 2] No such file or directory: 'waze_all_clean.csv'